In [1]:
%matplotlib inline
import plotly.io as pio
pio.renderers.default = "notebook"

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

# Interactive Hydrogen Evolution Reaction (HER) Fitting Analysis

This notebook provides an **interactive interface** for analyzing Hydrogen Evolution Reaction (HER) electrochemical data with:
- ⚡ Real-time parameter adjustment via interactive sliders
- 📊 Dynamic plot updates showing experimental vs. theoretical fits
- 🔬 Surface coverage analysis and Tafel slope calculations
- 🎯 Support for both simplified (2-step) and full (3-step) HER mechanisms

**How to use:**
1. Configure your data file path, electrode area, and corrections in the "Load and Process Experimental Data" section
2. Run all cells to fit your data
3. Use the interactive sliders to explore parameter sensitivity
4. Watch the four-panel dashboard update in real-time

For detailed documentation, see [README.md](README.md)

In [2]:
# Import Required Libraries
import numpy as np
import pandas as pd
from scipy import stats
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import ipywidgets as widgets
from ipywidgets import interact, FloatSlider, FloatLogSlider, Dropdown, HBox, VBox, Output
import random
from lmfit import Model, create_params

# Physical constants
F = 96485.3  # Faraday constant (C/mol)

print("✓ All libraries imported successfully")

✓ All libraries imported successfully


## Core Fitting Functions

In [3]:
def rnd():
    """Generate random initial parameter"""
    exp1 = random.randint(-15, -4)
    significand = round(random.uniform(0.1, 9), 2)
    return significand * 10 ** exp1

def HER_simplified_fitting(x, k1, k1r, k2, k2r, bbv, bbh, f1_val):
    """Simplified HER model"""
    vtotal = 2 * (((k1*k2*(1 - np.e**(2*f1_val*x)))*np.e**(-bbh*x*f1_val)) / 
                   (k1*np.e**((bbh - bbv)*f1_val*x) + k2 + np.e**(f1_val*x)*
                   (k1r*np.e**((bbh - bbv)*f1_val*x) + k2r)))
    return -F * vtotal

def Theta_VH(x, k1, k1r, k2, k2r, bbv, bbh, f1_val=38.92):
    """Calculate surface coverage for simplified model"""
    theta = (k1/np.e**(bbv*f1_val*x) + np.e**((1 - bbh)*f1_val*x)*k2r) / \
            (k1/np.e**(bbv*f1_val*x) + np.e**((1 - bbv)*f1_val*x)*k1r + 
             k2/np.e**(bbh*f1_val*x) + np.e**((1 - bbh)*f1_val*x)*k2r)
    return theta, 1 - theta

def Theta_Total(x, k1, k1r, k2, k2r, k3, k3r, bbv, bbh, f1_val=38.92):
    """Calculate surface coverage for full model"""
    k2r_calc = (k1*k2) / k1r
    k3r_calc = (k3*k1**2) / k1r**2
    A1 = -2*k3 + 2*k3r_calc
    B1 = (-np.e**((-bbv)*f1_val*x))*k1 - np.e**((1 - bbv)*f1_val*x)*k1r - \
         k2/np.e**(bbh*f1_val*x) - np.e**((1 - bbh)*f1_val*x)*k2r_calc - 4*k3r_calc
    C1 = k1/np.e**(bbv*f1_val*x) + np.e**((1 - bbh)*f1_val*x)*k2r_calc + 2*k3r_calc
    theta = (-B1 - np.sqrt(B1**2 - (4*A1*C1))) / (2*A1)
    return theta, 1 - theta

def rows_generator(df, start_idx=55):
    """Generate rolling window for Tafel slope calculation"""
    i = start_idx
    while (i + 10) <= df.shape[0]:
        yield df.iloc[i:(i+10):1, :]
        i += 1

def Tafel_Slopes(p, c):
    """Calculate Tafel slopes"""
    v1 = []
    df = pd.DataFrame({'Pot': p, 'Cur': c})
    for df_window in rows_generator(df):
        y = df_window['Pot'].values
        xz = np.log10(np.abs(df_window['Cur'].values))
        res = stats.linregress(xz, y)
        xm = y.mean()
        v1.append([np.abs(res.slope), xm])
    v2 = np.asarray(v1)
    return v2[:, 1], v2[:, 0]

## 2. Load and Process Experimental Data

This section loads your LSV (linear sweep voltammetry) data and applies necessary corrections:
- **Area Normalization**: Converts current to current density (mA/cm²)
- **Ohmic Drop Correction**: Removes resistive contribution from the system
- **Reference Potential Correction**: Adjusts to the desired reference electrode

**⚙️ Configuration**: Update the values below to match your experimental setup

In [4]:
# Load experimental data
file_path = r'/home/james/MEGA/Last_dance_Br/Mo2C_3/EF3_2/LSV_Mo2C_EF3_3(5).txt'
area_electrode = 0.5  # cm²
ohmic_drop = 6.43  # mV
ref_correction = 0.924  # V
temperature = 298.15  # K
gas_constant = 8.314  # J/(mol·K)

# Calculate f1
f1 = F / (temperature * gas_constant)
print(f"f1 calculated: {f1:.6f}")

# Load data
dframe = np.loadtxt(file_path, skiprows=1)
current_raw = dframe[:, 0]
potential_raw = dframe[:, 1]

# Process data
current = current_raw / area_electrode  # Normalize to current density
potential = potential_raw - (current_raw * ohmic_drop) + ref_correction  # Apply ohmic drop correction

print(f"Data loaded: {len(potential)} points")
print(f"Potential range: {potential.min():.3f} to {potential.max():.3f} V")
print(f"Current density range: {current.min():.3f} to {current.max():.3f} mA/cm²")

f1 calculated: 38.923897
Data loaded: 185 points
Potential range: -0.281 to -0.026 V
Current density range: -0.061 to -0.000 mA/cm²


## 3. Fit Data with Simplified or Full Model

**Select your model:**
- **`'simplified'`**: 2-step Volmer-Heyrovsky mechanism
  - H⁺ + e⁻ ⇌ H_ads (Volmer)
  - H_ads + H⁺ + e⁻ → H₂ (Heyrovsky)

- **`'full'`**: 3-step mechanism with Tafel pathway
  - H⁺ + e⁻ ⇌ H_ads (Volmer)  
  - H_ads + H⁺ + e⁻ ⇌ H₂ (Heyrovsky)
  - 2 H_ads → H₂ (Tafel)

The fitting process uses the Powell optimization method to find rate constants and symmetry factors that best fit your experimental data.

In [5]:
# Select model type
model_choice = 'simplified'  # Change to 'full' for full model

# Create wrapper functions
def HER_simplified_wrapper(x, k1, k1r, k2, k2r, bbv, bbh):
    return HER_simplified_fitting(x, k1, k1r, k2, k2r, bbv, bbh, f1)

def Hydrogen_Full_wrapper(x, k1, k1r, k2, k2r, k3, k3r, bbv, bbh):
    k2r_calc = (k1*k2) / k1r
    k3r_calc = (k3*k1**2) / k1r**2
    A1 = -2*k3 + 2*k3r_calc
    B1 = (-np.e**((-0.5)*f1*x))*k1 - np.e**((1 - 0.5)*f1*x)*k1r - k2/np.e**(0.5*f1*x) - np.e**((1 - 0.5)*f1*x)*k2r_calc - 4*k3r_calc
    C1 = k1/np.e**(0.5*f1*x) + np.e**((1 - 0.5)*f1*x)*k2r_calc + 2*k3r_calc
    theta1 = (-B1 - np.sqrt(B1**2 - (4*A1*C1))) / (2*A1)
    return -F*(k1*(1 - theta1))/np.e**(0.5*f1*x) - np.e**((1 - 0.5)*f1*x)*k2r_calc*(1 - theta1) - np.e**((1 - 0.5)*f1*x)*k1r*theta1 + (k2*theta1)/np.e**(0.5*f1*x)

# Prepare fitting
model_func = HER_simplified_wrapper if model_choice == 'simplified' else Hydrogen_Full_wrapper
HER_model = Model(model_func, independent_vars=['x'])

# Generate initial parameters
rand_params = np.array([rnd() for i in range(9)])

if model_choice == 'simplified':
    params = create_params(
        k1=dict(value=rand_params[0], max=1e-2, min=1e-20),
        k1r=dict(value=rand_params[1], max=1e-2, min=1e-20),
        k2=dict(value=rand_params[2], max=1e-2, min=1e-20),
        k2r=dict(value=rand_params[3], max=1e-2, min=1e-20),
        bbv=dict(value=0.5, min=0, max=1, vary=True),
        bbh=dict(value=0.5, min=0, max=1, vary=True)
    )
else:
    params = create_params(
        k1=dict(value=rand_params[0], max=1e-2, min=1e-20),
        k1r=dict(value=rand_params[1], max=1e-2, min=1e-20),
        k2=dict(value=rand_params[2], max=1e-2, min=1e-20),
        k2r=dict(expr='(k1*k2)/k1r'),
        k3=dict(value=rand_params[3], max=1e-2, min=1e-20),
        k3r=dict(expr='(k3*k1**2)/k1r**2'),
        bbv=dict(value=0.5, min=0, max=1, vary=True),
        bbh=dict(value=0.5, min=0, max=1, vary=True)
    )

# Perform fitting
print("Fitting data...")
result_model = HER_model.fit(
    current,
    params,
    x=potential,
    method='powell',
    nan_policy='omit'
)

print("Fitting completed!")
print(f"\nFitted parameters:")
for param_name in result_model.params:
    param = result_model.params[param_name]
    print(f"  {param_name}: {param.value:.6e} (min: {param.min}, max: {param.max})")

Fitting data...
Fitting completed!

Fitted parameters:
  k1: 7.469723e-07 (min: 1e-20, max: 0.01)
  k1r: 5.716062e-03 (min: 1e-20, max: 0.01)
  k2: 1.346314e-09 (min: 1e-20, max: 0.01)
  k2r: 1.000000e-02 (min: 1e-20, max: 0.01)
  bbv: 4.906597e-01 (min: 0, max: 1)
  bbh: 5.075392e-01 (min: 0, max: 1)


## 4. Create Interactive Parameter Controls

**Interactive Parameter Sliders**

The sliders below are automatically configured from the fitting results:
- **Log-scale sliders** for rate constants (k1, k1r, k2, k2r) with scientific notation
- **Linear sliders** for symmetry factors (bbv, bbh) from 0 to 1
- **Real-time labels** showing current value in scientific notation

📌 **Tip**: Adjust sliders to explore how parameters affect the fit quality

In [6]:
# Extract parameter limits from fitting results
param_sliders = {}
param_labels = {}
slider_ui = []

# Create sliders only for independent (non-expr) parameters
for param_name in result_model.params:
    param = result_model.params[param_name]
    # Skip parameters that are expressions (dependent)
    if getattr(param, 'expr', None):
        continue

    # Determine sensible min/max around fitted value
    fitted = float(param.value) if param.value is not None else 0.0
    if fitted == 0 or np.isnan(fitted):
        min_val = 1e-12
        max_val = 1.0
    else:
        # prefer positive ranges for rate constants
        min_val = param.min if param.min is not None else abs(fitted) * 1e-2
        max_val = param.max if param.max is not None else abs(fitted) * 1e2
        # ensure positive min for log sliders when possible
        if min_val <= 0:
            min_val = abs(fitted) * 1e-6 if fitted != 0 else 1e-12

    # Ensure ordering
    if min_val == max_val:
        max_val = min_val + abs(min_val) * 10 + 1.0

    # Choose a log slider for strictly positive ranges
    try:
        if (min_val > 0) and (fitted > 0):
            log_min = np.log10(min_val)
            log_max = np.log10(max_val)
            slider = FloatLogSlider(
                value=fitted,
                base=10,
                min=log_min,
                max=log_max,
                step=max((log_max - log_min) / 100.0, 0.01),
                description=param_name,
                style={'description_width': '120px'},
                layout=widgets.Layout(width='520px')
            )
        else:
            step = (max_val - min_val) / 100.0 if np.isfinite(max_val - min_val) and (max_val - min_val) != 0 else (abs(fitted) / 100 if fitted != 0 else 1e-6)
            slider = FloatSlider(
                value=fitted,
                min=min_val,
                max=max_val,
                step=step,
                description=param_name,
                style={'description_width': '120px'},
                layout=widgets.Layout(width='520px')
            )
    except Exception:
        # fallback
        slider = FloatSlider(value=fitted, min=-1.0, max=1.0, step=1e-6, description=param_name)

    # Force scientific readout on the slider if supported
    try:
        slider.readout = True
        slider.readout_format = '.3e'
    except Exception:
        pass

    # Create a label to show scientific formatted value
    label = widgets.Label(value=f"{fitted:.3e}", layout=widgets.Layout(width='80px'))

    # Update label when slider changes
    def _make_observer(lbl):
        def _obs(change):
            try:
                val = change['new']
                lbl.value = f"{val:.3e}"
            except Exception:
                pass
        return _obs

    obs = _make_observer(label)
    slider.observe(obs, names='value')

    # store
    param_sliders[param_name] = slider
    param_labels[param_name] = label
    slider_ui.append(HBox([slider, label]))

# Container will be displayed later in plotting cell
slider_box = VBox(slider_ui)

print("Parameter sliders created for independent parameters (dependent params will be computed)")

Parameter sliders created for independent parameters (dependent params will be computed)


## 5. Generate Interactive Plots with Dynamic Updates

**Four-Panel Analysis Dashboard**

The dashboard displays:
1. **Current vs. Potential** (top-left)
   - Blue dots: Experimental data
   - Red line: Theoretical fit from current parameters

2. **Surface Coverage θ** (top-right)
   - Green line: H-adsorbed coverage
   - Orange dashed line: Empty sites (1-θ)

3. **Tafel Slope Analysis** (bottom-left)
   - Blue squares: Experimental Tafel slope trend
   - Red line: Theoretical slope from model

4. **Current Parameters** (bottom-right)
   - Display of all active parameters in scientific notation
   - Updates in real-time as sliders move

**💡 Interaction**: Move any slider above to see the plots update instantly!

In [ ]:
# Function to calculate predictions with custom parameters
def calculate_predictions(k1, k1r, k2, k2r, bbv, bbh, k3=None, k3r=None):
    """Calculate model predictions with given parameters"""
    try:
        if model_choice == 'simplified':
            theoretical_current = HER_simplified_fitting(potential, k1, k1r, k2, k2r, bbv, bbh, f1)
            theta, theta_inv = Theta_VH(potential, k1, k1r, k2, k2r, bbv, bbh, f1)
        else:
            # For full model
            k2r_calc = (k1*k2) / k1r
            k3r_calc = (k3*k1**2) / k1r**2
            A1 = -2*k3 + 2*k3r_calc
            B1 = (-np.e**((-bbv)*f1*potential))*k1 - np.e**((1 - bbv)*f1*potential)*k1r - \
                 k2/np.e**(bbh*f1*potential) - np.e**((1 - bbh)*f1*potential)*k2r_calc - 4*k3r_calc
            C1 = k1/np.e**(bbv*f1*potential) + np.e**((1 - bbh)*f1*potential)*k2r_calc + 2*k3r_calc
            theta_pred = (-B1 - np.sqrt(np.abs(B1**2 - (4*A1*C1)))) / (2*A1 + 1e-50)
            theoretical_current = -F*(k1*(1 - theta_pred))/np.e**(bbv*f1*potential) - \
                                np.e**((1 - bbh)*f1*potential)*k2r_calc*(1 - theta_pred) - \
                                np.e**((1 - bbv)*f1*potential)*k1r*theta_pred + \
                                (k2*theta_pred)/np.e**(bbh*f1*potential)
            theta, theta_inv = Theta_Total(potential, k1, k1r, k2, k2r, k3, k3r, bbv, bbh, f1)
        
        return theoretical_current, theta, theta_inv
    except:
        return None, None, None

# Function to create interactive plots
def create_interactive_plots(k1, k1r, k2, k2r, bbv, bbh, k3=1e-10, k3r=1e-20):
    """Create three interactive Plotly subplots"""
    
    theoretical_current, theta, theta_inv = calculate_predictions(k1, k1r, k2, k2r, bbv, bbh, k3, k3r)
    
    if theoretical_current is None:
        print("Error in calculation")
        return
    
    # Create subplots
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=("Current vs Potential", "Surface Coverage (θ)", 
                       "Tafel Slope Analysis", "Parameter Values"),
        specs=[[{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}]]
    )
    
    # Plot 1: Current vs Potential
    fig.add_trace(
        go.Scatter(x=potential, y=current*1000, mode='markers', name='Experimental Data',
                  marker=dict(size=4, color='blue')),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(x=potential, y=theoretical_current*1000, mode='lines', name='Theoretical Fit',
                  line=dict(color='red', width=2)),
        row=1, col=1,
    )
    
    # Plot 2: Surface Coverage
    fig.add_trace(
        go.Scatter(x=potential, y=theta, mode='lines', name='θ (adsorbed H)',
                  line=dict(color='green', width=2)),
        row=1, col=2,
    )
    fig.add_trace(
        go.Scatter(x=potential, y=theta_inv, mode='lines', name='1-θ (empty site)',
                  line=dict(color='orange', width=2, dash='dash')),
        row=1, col=2,
    )
    
    # Plot 3: Tafel Slope
    try:
        pot1, tafel_exp = Tafel_Slopes(potential, current)
        pot2, tafel_theo = Tafel_Slopes(potential, theoretical_current)
        
        fig.add_trace(
            go.Scatter(x=pot1, y=tafel_exp, mode='markers', name='Experimental Tafel',
                      marker=dict(size=6, color='blue', symbol='square')),
            row=2, col=1,
        )
        fig.add_trace(
            go.Scatter(x=pot2, y=tafel_theo, mode='lines+markers', name='Theoretical Tafel',
                      line=dict(color='red', width=2), marker=dict(size=4)),
            row=2, col=1,
        )
    except:
        pass
    
    # Plot 4: Parameter Display
    param_text = f"""
    <b>Current Parameters:</b><br>
    k1 = {k1:.3e}<br>
    k1r = {k1r:.3e}<br>
    k2 = {k2:.3e}<br>
    k2r = {k2r:.3e}<br>
    bbv = {bbv:.4f}<br>
    bbh = {bbh:.4f}<br>
    """
    
    if model_choice == 'full':
        param_text += f"k3 = {k3:.3e}<br>k3r = {k3r:.3e}<br>"
    
    fig.add_annotation(
        text=param_text,
        xref="paper", yref="paper",
        x=0.5, y=0.5,
        showarrow=False,
        row=2, col=2,
        xanchor="center", yanchor="middle",
        font=dict(size=10),
        bgcolor="rgba(200,200,200,0.3)"
    )
    
    # Update layout
    fig.update_xaxes(title_text="Potential (V)", row=1, col=1)
    fig.update_yaxes(title_text="Current (mA/cm²)", row=1, col=1)
    
    fig.update_xaxes(title_text="Potential (V)", row=1, col=2)
    fig.update_yaxes(title_text="Surface Coverage", row=1, col=2)
    
    fig.update_xaxes(title_text="Potential (V)", row=2, col=1)
    fig.update_yaxes(title_text="Tafel Slope (mV/dec)", row=2, col=1)
    
    fig.update_layout(height=900, width=1200, hovermode='closest', showlegend=True)
    
    fig.show()

# Display parameter sliders
print("Adjust parameters using sliders below to update plots in real-time:")
for param_name, slider in param_sliders.items():
    print(f"{param_name}: {slider.value:.3e}")
    
# Create interactive plot widget
out = Output()

# Build a complete parameter dict from fitted values and slider overrides
def build_full_params():
    full = {}
    # start from fitted result
    for name, p in result_model.params.items():
        try:
            full[name] = float(p.value)
        except:
            full[name] = None
    # override with slider values
    for name, slider in param_sliders.items():
        full[name] = float(slider.value)
    # compute dependent params if missing
    if 'k2r' not in full or full.get('k2r') is None:
        try:
            full['k2r'] = (full['k1'] * full['k2']) / full['k1r']
        except:
            full['k2r'] = 1e-20
    if 'k3r' not in full or full.get('k3r') is None:
        try:
            full['k3r'] = (full.get('k3', 1e-10) * full['k1'] ** 2) / (full['k1r'] ** 2)
        except:
            full['k3r'] = 1e-20
    return full


def on_value_change(change):
    out.clear_output(wait=True)
    with out:
        params_dict = build_full_params()
        create_interactive_plots(k1=params_dict.get('k1', 1e-10),
                                 k1r=params_dict.get('k1r', 1e-10),
                                 k2=params_dict.get('k2', 1e-10),
                                 k2r=params_dict.get('k2r', 1e-20),
                                 bbv=params_dict.get('bbv', 0.5),
                                 bbh=params_dict.get('bbh', 0.5),
                                 k3=params_dict.get('k3', 1e-10),
                                 k3r=params_dict.get('k3r', 1e-20))

# Connect sliders to plot update
for slider in param_sliders.values():
    slider.observe(on_value_change, names='value')

# Display sliders
slider_box = VBox([param_sliders[name] for name in sorted(param_sliders.keys())])
display(slider_box)
display(out)

# Ensure sliders show fitted values
for name, slider in param_sliders.items():
    if name in result_model.params:
        try:
            slider.value = float(result_model.params[name].value)
        except:
            pass

# Initial plot using fitted parameters
with out:
    params_dict = build_full_params()
    create_interactive_plots(k1=params_dict.get('k1', 1e-10),
                             k1r=params_dict.get('k1r', 1e-10),
                             k2=params_dict.get('k2', 1e-10),
                             k2r=params_dict.get('k2r', 1e-20),
                             bbv=params_dict.get('bbv', 0.5),
                             bbh=params_dict.get('bbh', 0.5),
                             k3=params_dict.get('k3', 1e-10),
                             k3r=params_dict.get('k3r', 1e-20))

Adjust parameters using sliders below to update plots in real-time:
k1: 8.965e-07
k1r: 9.426e-03
k2: 1.357e-09
k2r: 1.000e-02
bbv: 4.925e-01
bbh: 5.081e-01


Output()